In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
sys.path.append(".")
import copy
import torch
import pickle
from datetime import datetime
import plotly
plotly.offline.init_notebook_mode(connected=True)

import robust_value_approx.utils as utils
import plotting_utils

import plotly.graph_objs as go

import pybullet as p
import robust_value_approx.constants as constants
import os
import inspect
import math
import robust_value_approx.ball_paddle_hybrid_linear_system as bp
import robust_value_approx.value_to_optimization as value_to_optimization
import time
import json

In [3]:
# x = [bally, paddley, ballvy, paddlevy]
# u = [paddlevy_dot]
# eef velocity limit is 1.7
# eef accel limit is 13.
N = 15
dtype = torch.float64
dt = .1
ball_target = torch.Tensor([2.]).type(dtype)
x_lo = torch.Tensor([0., 0., -100, -1.5]).type(dtype)
x_up = torch.Tensor([5., 1., 100, 1.5]).type(dtype)
u_lo = torch.Tensor([-5.]).type(dtype)
u_up = torch.Tensor([5.]).type(dtype)
sys_con = bp.get_ball_paddle_hybrid_linear_system_vertical
sys = sys_con(dtype, dt,
              x_lo, x_up,
              u_lo, u_up,
              cr=.9, collision_eps=.01,
              midpoint=True)
vf = value_to_optimization.ValueFunction(sys, N, x_lo, x_up, u_lo, u_up)
Q = torch.diag(torch.Tensor([1., 0., 0., 0.]).type(dtype))
Qt = torch.diag(torch.Tensor([100., 0., 0., 0.]).type(dtype))
R = torch.diag(torch.Tensor([.001]).type(dtype))
Rt = torch.diag(torch.Tensor([.001]).type(dtype))
xtraj = torch.Tensor(
    [ball_target[0], 0., 0., 0.]).type(dtype).unsqueeze(1).repeat(1, N-1)
vf.set_cost(Q=Q, R=R)
vf.set_terminal_cost(Qt=Qt, Rt=Rt)
vf.set_traj(xtraj=xtraj)

Using license file /Users/blandry/gurobi.lic
Academic license - for non-commercial use only


In [4]:
V = vf.get_value_function()

x0_lo = torch.Tensor([1.5, .15, -5., -1.]).type(vf.dtype)
x0_up = torch.Tensor([2., .15, 1., 5.]).type(vf.dtype)

In [5]:
# 0.788, -0.008, 1.735
bally0 = 1.735
paddley0 = .15
ballvy0 = 0.

# bally0 = 1.
# paddley0 = .15
# ballvy0 = -4.4

x0 = torch.Tensor([bally0, paddley0, ballvy0, 0.]).type(vf.dtype)
(x_traj, u_traj, alpha_traj) = vf.sol_to_traj(x0, *V(x0)[1:])
ttraj_val = torch.arange(0., vf.N*dt, dt, dtype=vf.dtype)

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=x_traj[0,:],
    name="ball"
))
fig.add_trace(go.Scatter(
    y=x_traj[1,:],
    name="paddle"
))
fig.show()

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=u_traj[0,:],
    name="paddle vel"
))
fig.show()

In [10]:
# load panda system

p.connect(p.GUI)
# p.connect(p.DIRECT)
p.setGravity(0, 0, constants.G)
currentdir = os.path.dirname(os.path.abspath(
                             inspect.getfile(inspect.currentframe())))
p.setAdditionalSearchPath(currentdir)
plane_id = p.loadURDF("../ball_panda_description/plane.urdf", [0, 0, 0])
# panda_id = p.loadURDF("../ball_panda_description/panda.urdf", [-.5, 0, 0])
# 0.788, -0.008, 1.735
panda_id = p.loadURDF("../ball_panda_description/panda.urdf", [-.788, 0, 0])
# panda_id = p.loadURDF("../ball_panda_description/panda.urdf", [-.688, 0, 0])

panda_num_joints = 7


error: Only one local in-process GUI/GUI_SERVER connection allowed. Use DIRECT connection mode or start a separate GUI physics server (ExampleBrowser, App_SharedMemoryPhysics_GUI, App_SharedMemoryPhysics_VR) and connect over SHARED_MEMORY, UDP or TCP instead.

In [67]:
# the true joint limits
# panda_ll = [-2.9671, -1.8326, -2.9671, -3.1416, -2.9671, -.0873, -2.9671]
# panda_ul = [2.9671, 1.8326, 2.9671, 0., 2.9671, 3.8223, 2.9671]
# joint limits to get nice IK solutions (more restricted)
# panda_ll = [-2.9671, -1.8326, -2.9671, -3.1416, -2.9671, -.0873, -2.9671]
# panda_ul = [2.9671, -.5, 2.9671, 0., 2.9671, 3.8223, 2.9671]

import numpy as np

panda_ll = (np.array([-2.9671, -1.8326, -2.9671, -3.1416, -2.9671, -.0873, -2.9671])*.8).tolist()
panda_ul = (np.array([2.9671, 1.8326, 2.9671, 0., 2.9671, 3.8223, 2.9671])*.8).tolist()

panda_jr = [2.*math.pi] * panda_num_joints
panda_rp = [0., -.785, 0., -2.356, 0., 1.571, .785]

In [68]:
# convert paddle traj to joint traj
q_traj = torch.zeros((panda_num_joints, vf.N), dtype=vf.dtype)
qdot_traj = torch.zeros((panda_num_joints, vf.N), dtype=vf.dtype)

for n in range(vf.N):
    pos = [0., 0., x_traj[1, n]]
    vel = [0., 0., x_traj[3, n]]
#     orn = p.getQuaternionFromEuler([.5*math.pi, 0., .5*math.pi])
    orn = p.getQuaternionFromEuler([0., 0., 0.])

    joint_poses = p.calculateInverseKinematics(panda_id,
                                               12,
                                               pos,
                                               orn,
                                               lowerLimits=panda_ll,
                                               upperLimits=panda_ul,
                                               jointRanges=panda_jr,
                                               restPoses=panda_rp,
                                               solver=p.IK_DLS)
    lin_jac, ang_jac = p.calculateJacobian(panda_id,
                                           12,
                                           [0., 0., 0.],
                                           joint_poses,
                                           [0.] * panda_num_joints,
                                           [0.] * panda_num_joints)
    lin_jac = torch.Tensor(lin_jac).type(vf.dtype)
    ee_vels = torch.Tensor(vel).type(vf.dtype)
    joint_vels = torch.pinverse(lin_jac) @ ee_vels
    q_traj[:, n] = torch.Tensor(joint_poses).type(vf.dtype)
    qdot_traj[:, n] = joint_vels

In [14]:
torch.max(torch.abs(qdot_traj))

tensor(1.4899, dtype=torch.float64)

In [ ]:
ttraj_val

In [69]:
# save trajectory to file as json
traj = dict()
traj['q'] = q_traj.t().numpy().tolist()
traj['qdot'] = qdot_traj.t().numpy().tolist()
traj['t'] = ttraj_val.numpy().tolist()
with open('panda_traj.json', 'w') as outfile:
    json.dump(traj, outfile)

In [ ]:
traj

In [ ]:
q_traj

In [70]:
n = 0

In [90]:
for i in range(panda_num_joints):
    p.resetJointState(panda_id, i, q_traj[i, n], 0.)
p.stepSimulation()
n = min(n+1, vf.N-1)

In [ ]:
# add ball
ball_id = p.loadURDF("../ball_panda_description/ball.urdf", [0, 0, bally0])
# p.resetBaseVelocity(ball_id, [0, 0, ballvy0])
# target_vis_id = p.createVisualShape(shapeType=p.GEOM_SPHERE,
#                                     rgbaColor=[0, 1, 1, .75],
#                                     specularColor=[.4, .4, 0],
#                                     radius=capture_size)
# target_col_id = p.createCollisionShape(shapeType=p.GEOM_SPHERE,
#                                        radius=capture_size)
# p.createMultiBody(baseVisualShapeIndex=target_vis_id,
#                   baseCollisionShapeIndex=target_col_id,
#                   basePosition=[ball_capture[0], 0., ball_capture[1]])
p.setPhysicsEngineParameter(restitutionVelocityThreshold=0.)
p.stepSimulation()
dt_sim = .001
p.setTimeStep(dt_sim)

In [ ]:
while True:
    p.stepSimulation()
    time.sleep(0.001)

In [ ]:
# interpolate it
ttraj_val_sim = torch.arange(0., N*dt, dt_sim, dtype=dtype)
ttraj_val_np = ttraj_val.detach().numpy()
xtraj_val_np = xtraj_val.detach().numpy()
utraj_val_np = utraj_val.detach().numpy()
ttraj_val_sim_np = ttraj_val_sim.detach().numpy()
xtraj_val_sim = torch.zeros(xtraj_val.shape[0], ttraj_val_sim.shape[0])
utraj_val_sim = torch.zeros(utraj_val.shape[0], ttraj_val_sim.shape[0])
for i in range(xtraj_val.shape[0]):
    xtraj_val_sim[i, :] = torch.Tensor(
        np.interp(ttraj_val_sim_np,
                  ttraj_val_np, xtraj_val_np[i, :])).type(dtype)
for i in range(utraj_val.shape[0]):
    utraj_val_sim[i, :] = torch.Tensor(
        np.interp(ttraj_val_sim_np,
                  ttraj_val_np, utraj_val_np[i, :])).type(dtype)
xtraj_sim = torch.zeros((sys.x_dim, ttraj_val_sim.shape[0]), dtype=dtype)

In [ ]:
# simulate it
for i in range(panda_num_joints):
    p.resetJointState(panda_id, i, q_traj[i], 0.)

for n in range(ttraj_val_sim.shape[0]):
    ball_pos = p.getBasePositionAndOrientation(ball_id)[0]
    (ballx, bally) = (ball_pos[0], ball_pos[2])
    ball_vel = p.getBaseVelocity(ball_id)[0]
    (ballvx, ballvy) = (ball_vel[0], ball_vel[2])
    paddle_state = p.getLinkState(panda_id, 12,
                                  computeLinkVelocity=1,
                                  computeForwardKinematics=1)
    (paddley, paddlevy) = (paddle_state[0][2], paddle_state[6][2])
    paddletheta = p.getEulerFromQuaternion(paddle_state[1])[0] - .5*math.pi
    x_current = torch.Tensor([ballx, bally, paddley, paddletheta,
                              ballvx, ballvy, paddlevy]).type(dtype)
    xtraj_sim[:, n] = x_current
    pos = [0., 0., xtraj_val_sim[2, n]]
    orn = p.getQuaternionFromEuler([.5*math.pi + xtraj_val_sim[3, n],
                                    0.,
                                    .5*math.pi])
    joint_poses = p.calculateInverseKinematics(panda_id,
                                               12,
                                               pos,
                                               orn,
                                               lowerLimits=panda_ll,
                                               upperLimits=panda_ul,
                                               jointRanges=panda_jr,
                                               restPoses=joint_poses,
                                               solver=p.IK_DLS)
    lin_jac, ang_jac = p.calculateJacobian(panda_id,
                                           12,
                                           [0., 0., 0.],
                                           joint_poses,
                                           [0.] * panda_num_joints,
                                           [0.] * panda_num_joints)
    lin_jac = torch.Tensor(lin_jac).type(dtype)
    ee_vels = torch.Tensor([0., 0., xtraj_val_sim[6, n]]).type(dtype)
    joint_vels = torch.pinverse(lin_jac) @ ee_vels
    p.setJointMotorControlArray(panda_id,
                                jointIndices=range(panda_num_joints),
                                controlMode=p.POSITION_CONTROL,
                                targetPositions=joint_poses,
                                targetVelocities=joint_vels,
                                forces=panda_efforts,
                                positionGains=1.*torch.ones(
                                    panda_num_joints),
                                velocityGains=1.*torch.ones(
                                    panda_num_joints))
    p.stepSimulation()
    time.sleep(0.001)

In [ ]:
    legend = []
    plt.plot(ttraj_val, xtraj_val[0, :], '*')
    legend.append('plan ballx')
    plt.plot(ttraj_val, xtraj_val[1, :], '*')
    legend.append('plan bally')
    plt.plot(ttraj_val, xtraj_val[2, :], '*')
    legend.append('plan paddle y')
    plt.plot(ttraj_val, xtraj_val[3, :], '*')
    legend.append('plan paddle theta')
    # plt.plot(ttraj_val, xtraj_val[6,:],'*')
    # legend.append('plan paddlev')
    plt.plot(ttraj_val_sim, xtraj_sim[0, :])
    legend.append('ballx')
    plt.plot(ttraj_val_sim, xtraj_sim[1, :])
    legend.append('bally')
    plt.plot(ttraj_val_sim, xtraj_sim[2, :])
    legend.append('paddle y')
    plt.plot(ttraj_val_sim, xtraj_sim[3, :])
    legend.append('paddle theta')
    # plt.plot(ttraj_val_sim, xtraj_sim[6,:])
    # legend.append('paddlev')
    plt.legend(legend)
    plt.show()